# Oracle 26ai를 Vector Store로 사용하여 Python/LangChain에서 RAG 구성하기

## Step 1: Indexing

### Step 1.1: Loading documents

In [ ]:
import bs4
import requests
import re
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import SeleniumURLLoader

urls = ["https://www.busan.com/view/busan/view.php?code=2025103023015638180",
       "https://www.yna.co.kr/view/AKR20251031055500003",
       "https://biz.sbs.co.kr/article/20000270136"]   

# 뉴스기사를 로드합니다
loader = WebBaseLoader(urls)
docs = loader.load()

for doc in docs:
    url = doc.metadata['source']
    html = requests.get(url).text
    soup = bs4.BeautifulSoup(html, "html.parser")

    # 메타데이타에 사이트이름을 추가합니다.
    meta_tag = soup.find("meta", attrs={"property": "og:site_name"}) or soup.find("meta", attrs={"name": "nate:site_name"})
    site_name = meta_tag["content"] if meta_tag and meta_tag.get("content") else None
    doc.metadata["site_name"] = site_name

    # 로딩한 텍스트를 정제합니다.
    clean_text = re.sub(r'\n\s*\n+', '\n', doc.page_content)  # 빈 줄 제거
    clean_text = re.sub(r'[ \t]+', ' ', clean_text)           # 중복 공백 제거
    clean_text = clean_text.strip()
    doc.page_content = clean_text    

In [ ]:
for doc in docs:
    print(f"metadata: {doc.metadata}")
    print(f"page_content: {doc.page_content}")

### Step 1.2: Splitting documents

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,       # chunk size (characters)
    chunk_overlap=50,     # chunk overlap (characters)
    add_start_index=True, # track index in original document
    separators=["\n\n", "\n", ".", " ", ""],
    keep_separator=True
)
all_splits = text_splitter.split_documents(docs)

print(f"텍스트 분할: {len(docs)}개 문서가 {len(all_splits)}개로 분할됨")

for index, split in enumerate(all_splits):
    print(f"#### index: {index}")
    print(f"     split: {split}")  

### Step 1.3: Storing documents

#### Text embedding

LangChain 문서 다음 링크에서 여러 embedding 모델에 따른 예시를 확인할 수 있습니다.
- https://python.langchain.com/docs/how_to/embed_text/#setup

여기서는 OCI Generative AI를 사용합니다.

자신이 사용하는 Compartment ID를 확인합니다.

-> [OCI Console에서 확인하러 가기](https://cloud.oracle.com/identity/compartments)

**확인 후 [.env](.env) 파일에 업데이트합니다.**

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
compartment_id = os.getenv("COMPARTMENT_ID")
print(f"compartment_id: {compartment_id}")

In [ ]:
from langchain_oci import OCIGenAIEmbeddings

# cohere.embed-v4.0 모델로 임베딩합니다.
embeddings_model = OCIGenAIEmbeddings(
    model_id="cohere.embed-v4.0",
    service_endpoint="https://inference.generativeai.ap-osaka-1.oci.oraclecloud.com",
    compartment_id=compartment_id,
    auth_type="INSTANCE_PRINCIPAL"
)

query_text = "젠슨황이 2025년 10월에 이재용, 정의선과 치맥한 가게 이름은?"
query_vector = embeddings_model.embed_query(query_text)

print(f"query_text: {query_text}")
print(f"query_vector: {query_vector}")

#### VectorStore: OracleVS vector store

LangChain 문서 다음 링크에서 여러 vector store에 따른 예시를 확인할 수 있습니다.
- https://python.langchain.com/docs/how_to/embed_text/#setup
- https://python.langchain.com/docs/integrations/vectorstores/

여기서는 Oracle Database 26ai를 vector store로 사용합니다.
- https://python.langchain.com/docs/integrations/vectorstores/oracle/

In [ ]:
# Free Container Image 사용하는 경우
import oracledb

username = "vector"
password = "vector"
dsn = "localhost:1521/FREEPDB1"

try:
    connection = oracledb.connect(user=username, password=password, dsn=dsn)

    cursor = connection.cursor()
    cursor.callproc("DBMS_APPLICATION_INFO.SET_CLIENT_INFO", ("oracle-devday/langchain-lab",))

    print("Connection successful!")
except Exception as e:
    print("Connection failed!")

In [ ]:
# OCI Autonomous AI Databases 사용하는 경우
# password, dsn을 업데이트합니다.
import oracledb

username = "vector"
password = "OraclePass123"
dsn = "yo64vsjrusqy4hks_low"

try:
    connection = oracledb.connect(
        config_dir="../wallet",
        user=username,
        password=password,
        dsn=dsn,
        wallet_location="../wallet",
        wallet_password=password)
    
    print("Oracle 26ai Autonomous Database Connection successful!")
except Exception as e:
    print("Oracle 26ai Autonomous Database Connection failed!")

In [ ]:
from langchain_oracledb.vectorstores import OracleVS
from langchain_community.vectorstores.utils import DistanceStrategy

vector_store = OracleVS(
    client=connection,        
    table_name="DOCUMENTS_COSINE",
    embedding_function=embeddings_model,
    distance_strategy=DistanceStrategy.COSINE,
)

**SQL Worksheet**로 DB에 접속하여 다음을 실행합니다. (**SQLcl**은 Python Environment와 충돌하여, 동작하지 않을 수 있습니다.)

```sql
desc DOCUMENTS_COSINE;
```

-> [SQL Worksheet로 이동하기](26ai-vector-search-langchain-lab.sql:7)

#### Storing documents

In [ ]:
# 분할된 청크들을 Vector Store에 저장합니다.
document_ids = vector_store.add_documents(documents=all_splits)

print(document_ids[:3])

**SQL Worksheet**로 DB에 접속하여 다음을 실행합니다.

```sql
SELECT id, text, to_char(METADATA), EMBEDDING FROM DOCUMENTS_COSINE;
```

-> [SQL Worksheet로 이동하기](26ai-vector-search-langchain-lab.sql:10)

## Step 2: Retrieval and Generation

### Step 2.1: Retrieve Test

In [ ]:
user_question = "젠슨황이 2025년 10월에 이재용, 정의선과 치맥한 가게 이름은?"

In [ ]:
retriever = vector_store.as_retriever()

In [ ]:
retrieved_docs = retriever.invoke(user_question)
print(len(retrieved_docs))

import json

for index, doc in enumerate(retrieved_docs):
    print(f"#### index: {index}")
    print(f"     metadata: {doc.metadata}")
    print(f"     page_content: {doc.page_content[:50]}")

### Step 2.1-1: Retrieve with Score

In [ ]:
from langchain_core.documents import Document
from langchain_core.runnables import chain
from typing import List

@chain
def retriever(query: str) -> List[Document]:
    docs, scores = zip(*vector_store.similarity_search_with_score(query))
    for doc, score in zip(docs, scores):
        doc.metadata["score"] = score

    return docs

In [ ]:
retrieved_docs = retriever.invoke(user_question)
print(len(retrieved_docs))

import json

for index, doc in enumerate(retrieved_docs):
    print(f"#### index: {index}")
    print(f"     metadata: {doc.metadata}")
    print(f"     page_content: {doc.page_content}")
    print(f"     score: {doc.metadata['score']}")

### Step 2.2: Prompt

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
Respond in Korean.
Always include the source of your answer, specifying the original document's title and link, in the format [출처: 문서 제목, 링크].

Question: {question} 
Context: {context} 
Answer:
""")

### Step 2.3: LLM

LangChain 문서 다음 링크에서 여러 chat model에 따른 예시를 확인할 수 있습니다.
- https://python.langchain.com/docs/integrations/chat/

여기서는 로컬 테스트를 위해 Ollama를 사용합니다.
- https://python.langchain.com/docs/integrations/chat/ollama/

In [ ]:
# 로컬 LLM 사용
from langchain_ollama import ChatOllama

llm = ChatOllama(
    #model="llama3.1",         #llama3.1:8b
    #model="llama3.2:latest",  #llama3.2:3b
    model="llama3.2:1b",
    temperature=0,
    # other params...
)

In [ ]:
# OCI Generative AI 사용
from langchain_oci import ChatOCIGenAI

llm = ChatOCIGenAI(
    #model_id="meta.llama-3.2-90b-vision-instruct",
    model_id="openai.gpt-oss-120b",
    service_endpoint="https://inference.generativeai.ap-osaka-1.oci.oraclecloud.com",
    compartment_id=compartment_id,
    model_kwargs={"temperature": 0},
    auth_type="INSTANCE_PRINCIPAL"
)

### Step 2.4: Answer
llama3.1:8b: AMD 1 OCPU - 6m 23s
llama3.2:1b: AMD 1 OCPU - 1m 2s

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

response = llm.invoke(user_question)
print(f"# LLM에 직접 질문하기")
print(f"# user_question: {user_question}")
print(f"# response: {response.content}")

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

question = user_question
response = chain.invoke(question)

print(f"\n# RAG를 사용하여 질문하기")
print(f"# user_question: {user_question}")
print(f"# response: {response}")

## Step 3: OracleVS

**SQL Worksheet**로 DB에 접속하여 다음을 실행합니다.
앞 코드 실행으로 인해 DB에서 실행한 SQL 쿼리를 확인하는 질의입니다.

```sql
SELECT sql_id, parsing_schema_name, sql_text
FROM v$sql
WHERE parsing_schema_name = 'VECTOR' and module like '%python'
ORDER BY last_active_time DESC;
````

-> [SQL Worksheet로 이동하기](26ai-vector-search-langchain-lab.sql:19)

실행한 SQL 쿼리에 대한 실행 계획을 확인합니다.

- 예시
```sql
EXPLAIN PLAN FOR
    SELECT id, text, metadata, vector_distance(embedding, :embedding, COSINE) as distance
      FROM DOCUMENTS_COSINE
  ORDER BY distance
     FETCH APPROX FIRST 4 ROWS ONLY;

SELECT * FROM TABLE(DBMS_XPLAN.DISPLAY(format => 'ALL'));
```

-> [SQL Worksheet로 이동하기](26ai-vector-search-langchain-lab.sql:25)

생성된 테이블에 INDEX가 없으므로, FULL Search하는 것을 볼 수 있습니다.

### Step 3.1: Index

#### Step 3.1.1: HNSW Index

In [ ]:
from langchain_oracledb.vectorstores import oraclevs

oraclevs.create_index(
    connection,
    vector_store,
    params={"idx_name": "documents_cosine_hnsw_idx", "idx_type": "HNSW"},
)

**SQL Worksheet**로 DB에 접속하여 다음을 실행합니다.

```sql
SELECT * FROM ALL_INDEXES WHERE table_name=UPPER('DOCUMENTS_COSINE');
```

인덱스가 생성된 것을 확인합니다.

-> [SQL Worksheet로 이동하기](26ai-vector-search-langchain-lab.sql:34)

Step 2.4과 동일한 내용으로 LLM에 다시 요청합니다.

In [ ]:
response = chain.invoke(question)

print(f"# user_question: {user_question}")
print(f"# response: {response}")

**SQL Worksheet**로 DB에 접속하여 다음을 실행합니다.
실행 계획이 달라졌는 지 확인합니다.

- 예시
```sql
EXPLAIN PLAN FOR
    SELECT id, text, metadata, vector_distance(embedding, :embedding, COSINE) as distance
      FROM DOCUMENTS_COSINE
  ORDER BY distance
     FETCH APPROX FIRST 4 ROWS ONLY;

SELECT * FROM TABLE(DBMS_XPLAN.DISPLAY(format => 'ALL'));
```

-> [SQL Worksheet로 이동하기](26ai-vector-search-langchain-lab.sql:37)

#### Step 3.1.2: IVF Index

In [ ]:
## 앞서 생성한 HNSW 인덱스를 삭제합니다.
from langchain_oracledb.vectorstores import oraclevs

oraclevs.drop_index_if_exists(
    connection,
    index_name="documents_cosine_hnsw_idx"
)

In [ ]:
from langchain_oracledb.vectorstores import oraclevs

oraclevs.create_index(
    connection,
    vector_store,
    params={"idx_name": "documents_cosine_ivf_idx", "idx_type": "IVF"},
)

### Step 3.2: Filtering

#### Step 3.2.1: Filtering - filter

필터링 없이 유사도 검색은 다음과 같이 수행합니다.

In [ ]:
# Similarity search without a filter
print("\nSimilarity search results without filter:")

retrieved_docs = vector_store.similarity_search(question, 2)

for index, doc in enumerate(retrieved_docs):
    print(f"#### index: {index}")
    print(f"     metadata: {doc.metadata}")
    print(f"     page_content: {doc.page_content}")

similarity_search() 함수를 통해 문서 retrieved_docs를 응답으로 받는 경우, chain 말고 각 문서의 page_content의 문자열을 context로 직접 사용하는 것도 가능합니다.

In [ ]:
context = "\n\n".join([doc.page_content for doc in retrieved_docs])
question = user_question

prompt = f"""You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
Respond in Korean.

Question: {question} 
Context: {context} 
Answer:
"""

response = llm.invoke(prompt)

print(f"# user_question: {user_question}")
print(f"# response: {response.content}")

metadata 칼럼 기준으로 아래와 같이 필터링 할 수 있습니다.

In [ ]:
# Similarity search with a filter
print("\nSimilarity search results with filter:")

#filter_dict = {"source": "https://www.busan.com/view/busan/view.php?code=2025103023015638180"};
filter_dict = {"site_name": "부산일보"};

retrieved_docs = vector_store.similarity_search(question, 3, filter=filter_dict);

for index, doc in enumerate(retrieved_docs):
    print(f"#### index: {index}")
    print(f"     metadata: {doc.metadata}")
    print(f"     page_content: {doc.page_content}")

**SQL Worksheet**로 DB에 접속하여 다음을 실행합니다.

실행된 쿼리를 확인합니다.

```sql
SELECT sql_id, parsing_schema_name, sql_text
FROM v$sql
WHERE parsing_schema_name = 'VECTOR' and module like '%python' and sql_text like '%DOCUMENTS_COSINE%'
ORDER BY last_active_time DESC;
```

-> [SQL Worksheet로 이동하기](26ai-vector-search-langchain-lab.sql:46)

실행된 쿼리를 확인해 보면 필터링이 **WHERE JSON_EXISTS**로 **Where 절 조건**이 추가된 것을 볼 수 있습니다.

```sql
SELECT id, text, metadata, vector_distance(embedding, :embedding, COSINE) as distance
FROM "DOCUMENTS_COSINE"
WHERE JSON_EXISTS(metadata, '$.source?(@ == $val)' PASSING :value0 AS "val")
ORDER BY distance
FETCH APPROX FIRST 3 ROWS ONLY
```

JSON 타입인 metadata에 대해서 추가적인 인덱스 설정이 필요합니다.

**SQL Worksheet**로 DB에 접속하여 다음을 실행합니다.

```sql
CREATE SEARCH INDEX metadata_json_search_idx
ON DOCUMENTS_COSINE (metadata)
FOR JSON;
```

-> [SQL Worksheet로 이동하기](26ai-vector-search-langchain-lab.sql:52)

인덱스 생성후 다시 확인해 보면 새로운 JSON 인덱스를 사용하는 것을 알 수 있습니다.

```sql
EXPLAIN PLAN FOR
  SELECT id, text, metadata, vector_distance(embedding, :embedding, COSINE) as distance
    FROM "DOCUMENTS_COSINE"
   WHERE JSON_EXISTS(metadata, '$.source?(@ == $val)' PASSING :value0 AS "val")
ORDER BY distance
   FETCH APPROX FIRST 3 ROWS ONLY;

SELECT * FROM TABLE(DBMS_XPLAN.DISPLAY(format => 'ALL'));
```

-> [SQL Worksheet로 이동하기](26ai-vector-search-langchain-lab.sql:57)

#### Step 3.2.2: Filtering - filter : score 함께 가져오기 (참고)

In [ ]:
# Similarity search with relevance score
print("\nSimilarity search with relevance score:")

retrieved_docs = vector_store.similarity_search_with_score(question, 2);

for index, doc_with_score in enumerate(retrieved_docs):
    print(f"#### index: {index}")
    #print(type(doc_with_score))
    doc = doc_with_score[0]
    score = doc_with_score[1]
    print(f"     metadata: {doc.metadata}")
    print(f"     page_content: {doc.page_content}")
    print(f"     score: {score}")

## Step 4: Existing Table with VECTOR type
이전 내용들은 LangChain에서 문서를 읽어 분리하고, 임베딩하고 DB 테이블에 저장하는 일련의 인덱싱 작업 후에 조회하는 과정이었습니다. OracleVS 패키지 또한 LangChain 연동에 맞춰 구현된 내용으로 그에 맞게 테이블 구조가 자동으로 만들어 진 상태였습니다.

여기서는 사용자가 이미 있는 테이블, 데이터에 대해 Oracle AI Database 26ai Vector Search 기능을 통해 유사도 질의를 하는 경우, LangChain에서 어떻게 사용할 까 하는 부분입니다. oracledb 패키지를 통해 Python에서 SQL 질의하고, 그 결과를 LangChain에 사용하는 langchain_core.documents.base.Document 클래스 형태로 변경하면 됩니다.

In [ ]:
from langchain_ollama import OllamaEmbeddings

#embeddings_model = OllamaEmbeddings(model="paraphrase-multilingual")

from langchain_oci import ChatOCIGenAI

# cohere.embed-v4.0 모델로 임베딩합니다.
embeddings_model = OCIGenAIEmbeddings(
    model_id="cohere.embed-v4.0",
    service_endpoint="https://inference.generativeai.ap-osaka-1.oci.oraclecloud.com",
    compartment_id=compartment_id,
    auth_type="INSTANCE_PRINCIPAL"
)

query_vector = embeddings_model.embed_query("다국어 메뉴판이 있는 여의도 맛집")
    
#print(type(query_vector)) # <class 'list'>
query_vector_str = str(query_vector)

In [ ]:
from langchain_core.documents import Document

cursor = connection.cursor()    

query = """
SELECT id, VECTOR_DISTANCE(VECTOR_DESCRIPTION, :query_vector, COSINE) AS vector_distance, business_type_registered, name, description
FROM RSTR_INFO
WHERE business_type_registered='한식'
ORDER BY vector_distance
FETCH FIRST 10 ROWS ONLY
"""
cursor.execute(query, {"query_vector": query_vector_str})
results = cursor.fetchall()

retrieved_docs = []

for result in results:
    metadata = {
        "id": result[0],
        "business_type_registered": result[2],
        "name": result[3]
        }
            
    doc = Document(
        page_content=(
            result[4]
            if result[4] is not None
            else ""
        ),
        metadata=metadata,
    )

    retrieved_docs.append(doc)

for index, doc in enumerate(retrieved_docs):
    print(f"#### index: {index}")
    print(f"     metadata: {doc.metadata}")
    print(f"     page_content: {doc.page_content}")

print("\n".join([f"{doc.metadata['name']}: {doc.page_content}" for doc in retrieved_docs]))    